In [254]:
from bs4 import BeautifulSoup
import pandas as pd
import requests
import json
import time 
from datetime import date
import csv
import numpy as np

## Caveat
The data extracted from the RMS website is unverified.

Verified data is published monthly and available as a PDF. However, the most recent verified data report is dated April 2017.

In [255]:
# Today's date. 
# Assume date is always DD/MM/YY format.
# Code hasn't been tested against D/M/YY date format. 

date_today = pd.datetime.today().date()
date_today = date_today.strftime('%d/%m/%y')
print('Today\'s date is', date_today)

Today's date is 24/06/18


In [256]:
# Keep for testing
#date_today = '20/06/18'

In [257]:
# M5 East data feeds
# Past 24h - scrape from web page
data_24h = 'http://www.rms.nsw.gov.au/projects/sydney-south/m5-east/autoupdate/m5eastcurrentairqualitydata/past24hours.html'
# Past 4 weeks - CSV file
data_4w = 'http://www.rms.nsw.gov.au/projects/sydney-south/m5-east/autoupdate/m5eastcurrentairqualitydata/m5eastair.csv'

## Clean the 24 hour data

In [258]:
# Collect the data tables
r = requests.get(data_24h)
soup = BeautifulSoup(r.content, "lxml")
soup = soup.find_all('div', class_='pane-right')
table = soup[0].find_all('table')

In [259]:
# Transform table into a data frame
df = pd.read_html(str(table))

In [260]:
#Add station IDs. Note: tables are repeated three times. (See note below.)
df[0].insert(0, 'Station', 'X1')
df[3].insert(0, 'Station', 'U1')
df[6].insert(0, 'Station', 'T1')
df[9].insert(0, 'Station', 'CBMS')

In [261]:
# HTML tables are repeated three times. 
# They seem to be repeated and structured differently for mobile devices, etc.
# Create a new list without repeated tables.
df_remove_dups = [] 
df_remove_dups.append(df[0])
df_remove_dups.append(df[3])
df_remove_dups.append(df[6])
df_remove_dups.append(df[9])
df = pd.concat(df_remove_dups)

In [262]:
# Add column names
df.columns = ['Station', 'Date', 'Time', 'CO_ppm', 'NO2', 'PM10', 'wind_dir', 'wind_speed', 'PM10_daily']

In [263]:
# Remove yesterday's data as it is also included in the 4 week data.
# Last entry is recorded at 11pm the previous day.
df_remove_yesterday = df[df['Date'] == date_today]
df = df_remove_yesterday

In [264]:
# Tidy the time range to start hour.
# Create temporary column then drop it
df['Time'], df['temp_hour_end'] = df['Time'].str.split(' - ', 1).str
df.drop('temp_hour_end', axis=1, inplace=True)

## Clean the 4-week data

In [265]:
# Import the data
df_4week = pd.read_csv(data_4w)
df_4week.columns = ['Station', 'Date', 'Time', 'CO_ppm', 'NO2', 'PM10', 'wind_dir', 'wind_speed', 'PM10_daily']

## Combine the dataframes

In [266]:
# Combine today's data with teh previosu four weeks' data.
df_combined = []
df_combined.append(df)
df_combined.append(df_4week)
df = pd.concat(df_combined)

In [267]:
# Temporary calculation to create timestamp
df.insert(1, 'Timestamp', '')
df['temp_date_with_time'] = df['Date'] + ' ' + df['Time']
df['Timestamp'] = pd.to_datetime(df['temp_date_with_time'], format='%d/%m/%y %I%p')
df.drop('temp_date_with_time', axis=1, inplace=True)
df.drop('Date', axis=1, inplace=True)
df.drop('Time', axis=1, inplace=True)
df.reset_index(drop=True)

,Station,Timestamp,CO_ppm,NO2,PM10,wind_dir,wind_speed,PM10_daily
0,X1,2018-06-24 00:00:00,0.24,6.2,10.1,SW,11.0,NaN
1,X1,2018-06-24 01:00:00,0.20,12.3,8.8,W,7.0,NaN
2,X1,2018-06-24 02:00:00,0.16,28.8,8.0,WNW,12.0,NaN
3,X1,2018-06-24 03:00:00,0.15,24.6,7.1,WNW,8.0,NaN
4,X1,2018-06-24 04:00:00,0.14,30.8,5.8,NW,8.0,NaN
5,X1,2018-06-24 05:00:00,0.13,43.1,7.2,NW,7.0,NaN
6,X1,2018-06-24 06:00:00,0.14,41.1,8.1,NW,7.0,NaN
7,X1,2018-06-24 07:00:00,0.18,37.0,6.1,NW,9.0,NaN
8,X1,2018-06-24 08:00:00,0.19,26.7,4.6,WNW,7.0,NaN
9,X1,2018-06-24 09:00:00,0.20,22.6,8.1,WNW,6.0,NaN


In [268]:
# Add extra columns to pre for Lane Cove table merge
df.insert(2, 'CO_3min', '')
df.insert(3, 'CO_15min', '')
df.insert(4, 'CO_30min', '')
df.head(5)

,Station,Timestamp,CO_3min,CO_15min,CO_30min,CO_ppm,NO2,PM10,wind_dir,wind_speed,PM10_daily
1,X1,2018-06-24 00:00:00,,,,0.24,6.2,10.1,SW,11.0,NaN
2,X1,2018-06-24 01:00:00,,,,0.20,12.3,8.8,W,7.0,NaN
3,X1,2018-06-24 02:00:00,,,,0.16,28.8,8.0,WNW,12.0,NaN
4,X1,2018-06-24 03:00:00,,,,0.15,24.6,7.1,WNW,8.0,NaN
5,X1,2018-06-24 04:00:00,,,,0.14,30.8,5.8,NW,8.0,NaN


## Calculate the M5 East AQI
Guide to calculating the AQI:
http://www.environment.nsw.gov.au/topics/air/understanding-air-quality-data/air-quality-index

In [269]:
# Calc -- AQI = (polutant reading / standard) * 100
# AQI is highest of the readings
#
# M5 East does not monitor
# O3 Ozone -- 0.10 ppm over 1 hr
# PM2.5 Small particle -- 25 over 24 hr

# CO -- 9ppm over 8 hours
df['AQI_CO'] = ( df['CO_ppm'] / 9 ) * 100

# NO2 -- 0.12 ppm over 1 hr
# This is approximated - published AQI is only recorded in the verified data
df['AQI_NO2'] = ( df['NO2'] / 2000 ) * 100 

# PM10 -- 50 over 24 hr
df['AQI_PM10'] = ( df['PM10_daily'] / 50 ) * 100

In [270]:
# Pick highest AQI, round it up and add as a new column
aqindex = df[['AQI_CO', 'AQI_NO2', 'AQI_PM10']].apply(max, axis=1)
df['AQI'] = aqindex

In [271]:
# Fill AQI NaN
df['AQI'] = df['AQI'].fillna(-1)

In [272]:
# Round up AQI
aqi = df['AQI']
aqi = np.ceil(aqi).astype(int)
df['AQI'] = aqi

In [273]:
# Assign M5E to a copy
df_M5E = df

## Clean Lane Cove Tunnel data

In [274]:
# Site URLs
url = "https://www.lanecovetunnel.com/"
path = "cott.csv"
url = url + path

In [275]:
# Read CSV
df = pd.read_csv(url)

In [276]:
# Add table headers
# Data in column index 4 is undocumented
df.columns = ['Date_time', 'CO_3min', 'CO_15min', 'CO_30min','Undocumented']

In [277]:
# Insert columns
df.insert(0, 'Station', 'LCTINT')
df.insert(1, 'Timestamp', '')

# Insert columns
df.insert(7, 'CO_ppm', '')
df.insert(8, 'NO2', '')
df.insert(9, 'PM10', '')
df.insert(10, 'wind_dir', '')
df.insert(11, 'wind_speed', '')
df.insert(12, 'PM10_daily', '')
df.insert(13, 'AQI_CO', '')
df.insert(14, 'AQI_NO2', '')
df.insert(15, 'AQI_PM10', '')

In [278]:
# Create timestamp
df['Timestamp'] = pd.to_datetime(df['Date_time'], format='%d/%m/%Y %H:%M')

In [279]:
# modify df. Drop unneccessary columns
df.drop('Date_time', axis=1, inplace=True)
df.drop('Undocumented', axis=1, inplace=True)

## Calculate Lane Cove AQI

In [280]:
# Calc -- AQI = (polutant reading / standard) * 100
# AQI is highest of the readings
# 
# CO level is the only publicly available value 
# Use operators maximum as the 'standard'
# CO over 3mins limit is 200

# CO -- 200ppm over 3 minutes
df['AQI_CO'] = ( df['CO_3min'] / 200 ) * 100

In [281]:
# Pick highest AQI, round it up and add as a new column
aqindex = df[['AQI_CO']].apply(max, axis=1)
df['AQI'] = aqindex
aqi = df['AQI'].astype(int)
df['AQI'] = aqi

## Combine M5 East and Lane Cove Tunnel data frames

In [282]:
## Combine M5 East and Lane Cove Tunnel data frames
df_motorways = []
df_motorways.append(df)
df_motorways.append(df_M5E)
df_m = pd.concat(df_motorways)

In [283]:
df_m.reset_index(drop=True)
#df_m

,Station,Timestamp,CO_3min,CO_15min,CO_30min,CO_ppm,NO2,PM10,wind_dir,wind_speed,PM10_daily,AQI_CO,AQI_NO2,AQI_PM10,AQI
0,LCTINT,2018-06-24 00:00:00,2,0,0,,,,,,,1.000000,,,1
1,LCTINT,2018-06-24 01:00:00,2,0,0,,,,,,,1.000000,,,1
2,LCTINT,2018-06-24 02:00:00,2,0,0,,,,,,,1.000000,,,1
3,LCTINT,2018-06-24 03:00:00,2,0,0,,,,,,,1.000000,,,1
4,LCTINT,2018-06-24 04:00:00,2,0,0,,,,,,,1.000000,,,1
5,LCTINT,2018-06-24 05:00:00,2,0,0,,,,,,,1.000000,,,1
6,LCTINT,2018-06-24 06:00:00,2,0,0,,,,,,,1.000000,,,1
7,LCTINT,2018-06-24 07:00:00,2,0,0,,,,,,,1.000000,,,1
8,LCTINT,2018-06-24 08:00:00,4,1,0,,,,,,,2.000000,,,2
9,LCTINT,2018-06-24 09:00:00,5,1,0,,,,,,,2.500000,,,2


## Calculate AQI band

In [284]:
# Fill AQI NaN
df_m['AQI'] = df_m['AQI'].fillna(-1)

In [285]:
df_m.insert(15, 'AQI_level', -1)
df_m.insert(16, 'AQI_description', '-')

In [286]:
#df_m

In [287]:
for index, row in df_m.iterrows():
    aqi = row['AQI']
    if aqi > 200:
        AQI_level = 6
        AQI_description = 'Hazardous'
        #break
    elif aqi > 149:
        AQI_level = 5
        AQI_description = 'Very poor'
        #break
    elif aqi > 99:
        AQI_level = 4
        AQI_description = 'Poor'
        #break
    elif aqi > 66:
        AQI_level = 3
        AQI_description = 'Fair'
        #break
    elif aqi > 33:
        AQI_level = 2
        AQI_description = 'Good'
        #break
    elif aqi > -1:
        AQI_level = 1
        AQI_description = 'Very good'
        #break
    else:
        AQI_level = 0
        AQI_description = 'AQI unavailable'

    df_m.loc[index, 'AQI_level'] = AQI_level
    df_m.loc[index, 'AQI_description'] = AQI_description

#    print(index, aqi, AQI_description, AQI_level)

In [288]:
#df_m[580:590]

## Add sensor locations

In [289]:
# Open local file. Sensor locations added to OSM then extracted.
data_aq_sites = "./labels_air_quality_monitors.csv"
sites = pd.read_csv(data_aq_sites)

In [290]:
# Merge dataframes
df_sites = pd.merge(df_m, sites, on='Station', how='left')
df_m = df_sites

## Export data
Create a CSV file and write to it. 

In [291]:
# Write to CSV
output = './AQI-SydneyMotorways.csv'
ofile = open(output, 'w')
df_m.to_csv(output)
ofile.close()